# 导出并载入词库

极点五笔集成了词库导出功能，将导出的词库 `freeime.txt` 载入 python。但是有几点需要注意：

* 编码格式，极点五笔的导出词库编码经测试是 'utf16'
* 在转换之前要去除词库文件前面的头信息
* 在部分词库文件中，后面有以编码 `zz` 开头词组，是极点的特殊符号及标点，要去掉
* 在词库中，有部分词为极点中如 `time`, `date` 之类的功能，要去掉

In [32]:
# 极点词库文件
freeime = r'c:\users\way\documents\freeime.txt'

# 读取文件
lines = open(freeime, 'r', encoding='utf16').readlines()

#去掉头部和最后的 zz 标点部分
start = lines.index('[Text]\n') + 1    # [Text]\n 的下一行

# 去掉 zz 开始之后为特殊标点
zz = [i for i in lines if i.startswith('zz')]
if zz:
    stop = lines.index(zz[0])
else:
    stop = len(lines)

lines = lines[start:stop]

#去掉特殊命令部分（主要标志是有 $ ）符号，如果 date
special = [i for i in lines if '$' in i]
for l in special:
    s = [i for i in l.split(' ') if '$' not in i]
    if len(s)>1:
        lines[lines.index(l)] = ' '.join(s)
    else:
        lines.remove(l)

# 转换为精灵五笔格式

精灵五笔本身自带词库导入工具，下面整理的格式可以直接导入。但是在测试过程中，发现某些时候精灵五笔是按倒序导入的，但是在另外一些情况下，又是按顺序导入。暂时没找到原因和规律，因此顺序的问题需要试几次。

In [80]:
jl_sys = []
jl_gb = []

jl_sys_head = """---config@码表分类=主码-系统码表
---config@码表别名=系统词库\n"""
jl_gb_head = '''---config@码表分类=主码-次显码表
---config@码表别名= GB18030\n'''

# jl_sys.append(jl_sys_head)
# jl_gb.append(jl_gb_head)

for l in lines:
    words = l.split()
    code = words[0]
    for w in words[1:]:
        if not w.startswith('~'):
            #注意注意：在精灵码表里，是字在先编码在后
            jl_sys.append('{1}\t{0}\n'.format(code, w.strip('^')))
        elif w.startswith('~'):
            jl_gb.append('{1}\t{0}\n'.format(code, w.strip('~')))

In [81]:
with open('jl-sys.txt', 'w', encoding='utf8') as f:
    f.write(jl_sys_head)
    f.write(''.join(jl_sys[::-1]))  #此处顺序要测试
#     f.write(''.join(jl_sys))
with open('jl-gb.txt', 'w', encoding='utf8') as f:
    f.write(jl_gb_head)
    f.write(''.join(jl_gb[::-1]))
#     f.write(''.join(jl_gb))

# 转换为 fcitx 格式

fcitx 在 linux 下有 `mb2txt`命令将 fcitx 词库导出为文本文件，`txt2mb`导入为词库文件

In [86]:
example = open('wbx.txt', encoding='utf8').readlines(500)
fc_head = example[:example.index('[Data]\n')+1]

fc = []

for l in lines:
    words = l.split()
    code = words[0]
    for w in words[1:]:
        if not w.startswith('~'):
            # 在 fcitx 编码里面是编码先，字在后
            fc.append('{0} {1}\n'.format(code, w.strip('^')))

In [87]:
with open('fc-jd.txt', 'w', encoding='utf8') as f:
    f.write(''.join(fc_head))
    f.write(''.join(fc))